# Exploración de Datos

Este notebook muestra cómo conectarse a la base de datos PostgreSQL y realizar una exploración inicial de los datos.

## Configuración del entorno

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import json

# Configurar visualizaciones
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)

# Configurar opciones de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

## Conexión a la base de datos

Primero, establecemos la conexión con nuestra base de datos PostgreSQL.

In [2]:
# Conectar a la base de datos
DATABASE_URL = "postgresql://postgres:postgres@db:5432/dataengineering"
engine = create_engine(DATABASE_URL)
connection = engine.connect()

print("Conexión establecida con la base de datos.")

Conexión establecida con la base de datos.


## Verificar las tablas disponibles

Vamos a verificar qué tablas están disponibles en nuestra base de datos.

In [3]:
# Consultar las tablas disponibles
query = text("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'
""")

tables = pd.read_sql(query, connection)
tables

,table_name
0,raw_data
1,processed_data
2,final_data
3,cache_config
4,databasechangelog
...,...
114,Genre
115,MediaType
116,Artist
117,Album


## Cargar datos de ejemplo

Para este ejemplo, vamos a crear algunos datos de prueba y cargarlos en la tabla `raw_data`.

In [4]:
# Crear datos de ejemplo
import datetime

# Crear un DataFrame de ejemplo
data = {
    'id': range(1, 11),
    'valor': np.random.randint(10, 100, 10),
    'categoria': np.random.choice(['A', 'B', 'C'], 10),
    'fecha': [datetime.datetime.now() - datetime.timedelta(days=i) for i in range(10)]
}

df = pd.DataFrame(data)
df

,id,valor,categoria,fecha
0,1,68,B,2025-07-07 22:54:46.906768
1,2,89,A,2025-07-06 22:54:46.906778
2,3,59,C,2025-07-05 22:54:46.906780
3,4,51,B,2025-07-04 22:54:46.906782
4,5,60,A,2025-07-03 22:54:46.906783
5,6,84,B,2025-07-02 22:54:46.906784
6,7,43,A,2025-07-01 22:54:46.906786
7,8,19,B,2025-06-30 22:54:46.906787
8,9,54,B,2025-06-29 22:54:46.906788
9,10,45,A,2025-06-28 22:54:46.906790


In [5]:
# Insertar datos en la tabla raw_data
for _, row in df.iterrows():
    # Convertir el timestamp a string para que sea serializable
    row_dict = row.to_dict()
    
    # Convertir el objeto Timestamp a string si existe
    if 'fecha' in row_dict:
        row_dict['fecha'] = str(row_dict['fecha'])
    
    # Convertir a formato JSON
    data_json = json.dumps(row_dict)
    
    # Preparar la consulta de inserción
    insert_query = text("""
        INSERT INTO raw_data (timestamp, source, data)
        VALUES (:timestamp, :source, :data)
    """)
    
    # Ejecutar la consulta
    connection.execute(insert_query, {
        'timestamp': datetime.datetime.now(),
        'source': 'ejemplo_notebook',
        'data': data_json
    })

print("Datos insertados correctamente en la tabla raw_data.")

Datos insertados correctamente en la tabla raw_data.


## Consultar datos de la tabla raw_data

Ahora vamos a consultar los datos que acabamos de insertar.

In [6]:
# Consultar datos de raw_data
query = text("SELECT id, timestamp, source, data FROM raw_data LIMIT 10")
raw_data = pd.read_sql(query, connection)
raw_data

,id,timestamp,source,data
0,1,2025-05-18 23:51:50.303212+00:00,ejemplo_notebook,"{'id': 1, 'fecha': '2025-05-18 23:39:46.137472..."
1,2,2025-05-18 23:51:50.323238+00:00,ejemplo_notebook,"{'id': 2, 'fecha': '2025-05-17 23:39:46.137484..."
2,3,2025-05-18 23:51:50.325509+00:00,ejemplo_notebook,"{'id': 3, 'fecha': '2025-05-16 23:39:46.137487..."
3,4,2025-05-18 23:51:50.328453+00:00,ejemplo_notebook,"{'id': 4, 'fecha': '2025-05-15 23:39:46.137489..."
4,5,2025-05-18 23:51:50.330390+00:00,ejemplo_notebook,"{'id': 5, 'fecha': '2025-05-14 23:39:46.137491..."
5,6,2025-05-18 23:51:50.332393+00:00,ejemplo_notebook,"{'id': 6, 'fecha': '2025-05-13 23:39:46.137493..."
6,7,2025-05-18 23:51:50.334153+00:00,ejemplo_notebook,"{'id': 7, 'fecha': '2025-05-12 23:39:46.137495..."
7,8,2025-05-18 23:51:50.336348+00:00,ejemplo_notebook,"{'id': 8, 'fecha': '2025-05-11 23:39:46.137496..."
8,9,2025-05-18 23:51:50.338515+00:00,ejemplo_notebook,"{'id': 9, 'fecha': '2025-05-10 23:39:46.137498..."
9,10,2025-05-18 23:51:50.340399+00:00,ejemplo_notebook,"{'id': 10, 'fecha': '2025-05-09 23:39:46.13750..."


## Procesar los datos

Vamos a procesar los datos y almacenarlos en la tabla `processed_data`.

In [7]:
# Procesar los datos
for _, row in raw_data.iterrows():
    # Extraer el JSON de la columna data
    data = json.loads(row['data'])
    
    # Realizar alguna transformación (ejemplo: duplicar el valor)
    if 'valor' in data:
        data['valor_duplicado'] = data['valor'] * 2
    
    # Convertir a formato JSON
    processed_data_json = json.dumps(data)
    
    # Insertar en la tabla processed_data
    insert_query = text("""
        INSERT INTO processed_data (raw_data_id, data)
        VALUES (:raw_data_id, :data)
    """)
    
    connection.execute(insert_query, {
        'raw_data_id': row['id'],
        'data': processed_data_json
    })

print("Datos procesados correctamente y almacenados en la tabla processed_data.")

TypeError: the JSON object must be str, bytes or bytearray, not dict

## Consultar datos procesados

Ahora vamos a consultar los datos procesados.

In [ ]:
# Consultar datos de processed_data
query = text("""
    SELECT p.id, p.raw_data_id, p.processed_at, p.data
    FROM processed_data p
    JOIN raw_data r ON p.raw_data_id = r.id
    LIMIT 10
""")

processed_data = pd.read_sql(query, connection)
processed_data

## Análisis exploratorio

Vamos a realizar un análisis exploratorio básico de los datos procesados.

In [ ]:
# Extraer los datos JSON en un DataFrame
data_list = []

for _, row in processed_data.iterrows():
    data_dict = json.loads(row['data'])
    data_dict['processed_id'] = row['id']  # Agregar el ID del registro procesado
    data_list.append(data_dict)

# Crear DataFrame
analysis_df = pd.DataFrame(data_list)
analysis_df

In [ ]:
# Análisis estadístico básico
analysis_df.describe()

In [ ]:
# Visualización: Distribución de valores
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(analysis_df['valor'], kde=True)
plt.title('Distribución de Valores Originales')

plt.subplot(1, 2, 2)
sns.histplot(analysis_df['valor_duplicado'], kde=True)
plt.title('Distribución de Valores Duplicados')

plt.tight_layout()
plt.show()

In [ ]:
# Visualización: Valores por categoría
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.boxplot(x='categoria', y='valor', data=analysis_df)
plt.title('Valores por Categoría')

plt.subplot(1, 2, 2)
sns.boxplot(x='categoria', y='valor_duplicado', data=analysis_df)
plt.title('Valores Duplicados por Categoría')

plt.tight_layout()
plt.show()

## Generar insights y cargar en la tabla final_data

Finalmente, vamos a generar algunos insights y cargarlos en la tabla `final_data`.

In [ ]:
# Calcular métricas por categoría
category_metrics = analysis_df.groupby('categoria').agg({
    'valor': ['mean', 'min', 'max', 'std'],
    'valor_duplicado': ['mean', 'min', 'max', 'std']
})

category_metrics

In [ ]:
# Insertar insights en la tabla final_data
for processed_id in processed_data['id']:
    # Obtener los datos procesados
    processed_row = processed_data[processed_data['id'] == processed_id].iloc[0]
    processed_data_dict = json.loads(processed_row['data'])
    
    # Extraer la categoría
    categoria = processed_data_dict.get('categoria', 'Desconocida')
    
    # Calcular métricas
    if 'valor' in processed_data_dict:
        valor = processed_data_dict['valor']
        valor_duplicado = processed_data_dict.get('valor_duplicado', 0)
        
        # Crear métricas
        metrics = {
            'valor_original': valor,
            'valor_duplicado': valor_duplicado,
            'diferencia': valor_duplicado - valor,
            'categoria': categoria
        }
        
        # Generar insights
        insights = f"El valor {valor} de la categoría {categoria} fue duplicado a {valor_duplicado}. "
        
        # Comparar con el promedio de la categoría
        if categoria in category_metrics.index:
            avg_valor = category_metrics.loc[categoria, ('valor', 'mean')]
            if valor > avg_valor:
                insights += f"El valor está por encima del promedio de la categoría ({avg_valor:.2f})."
            else:
                insights += f"El valor está por debajo del promedio de la categoría ({avg_valor:.2f})."
        
        # Insertar en la tabla final_data
        insert_query = text("""
            INSERT INTO final_data (processed_data_id, metrics, insights)
            VALUES (:processed_data_id, :metrics, :insights)
        """)
        
        connection.execute(insert_query, {
            'processed_data_id': processed_id,
            'metrics': json.dumps(metrics),
            'insights': insights
        })

print("Insights generados y almacenados en la tabla final_data.")

## Consultar datos finales

Por último, vamos a consultar los datos finales con sus insights.

In [ ]:
# Consultar datos de final_data
query = text("""
    SELECT f.id, f.processed_data_id, f.created_at, f.metrics, f.insights
    FROM final_data f
    JOIN processed_data p ON f.processed_data_id = p.id
    LIMIT 10
""")

final_data = pd.read_sql(query, connection)
final_data

In [ ]:
# Extraer métricas para análisis
metrics_list = []

for _, row in final_data.iterrows():
    metrics_dict = json.loads(row['metrics'])
    metrics_dict['final_id'] = row['id']
    metrics_dict['insights'] = row['insights']
    metrics_list.append(metrics_dict)

# Crear DataFrame
metrics_df = pd.DataFrame(metrics_list)
metrics_df

In [ ]:
# Visualización final: Comparación de valores originales y duplicados
plt.figure(figsize=(10, 6))

# Crear datos para el gráfico
x = range(len(metrics_df))

plt.bar(x, metrics_df['valor_original'], width=0.4, align='edge', label='Valor Original')
plt.bar([i+0.4 for i in x], metrics_df['valor_duplicado'], width=0.4, align='edge', label='Valor Duplicado')

plt.xlabel('Registro')
plt.ylabel('Valor')
plt.title('Comparación de Valores Originales y Duplicados')
plt.legend()
plt.xticks([i+0.2 for i in x], metrics_df['final_id'])

plt.tight_layout()
plt.show()

## Cerrar la conexión

Finalmente, cerramos la conexión a la base de datos.

In [ ]:
# Cerrar la conexión
connection.close()
print("Conexión cerrada.")

# Titulo de ejemplo custom: nueva tarea ejemplo.

In [ ]:
# este es un ejemplo de notebook con codigo de comentarios.